In [7]:
import pandas as pd

output_correct = pd.DataFrame()

In [8]:
import os

In [9]:
list_file = list(os.walk('./part'))[0][2]
if '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
if '0.null.txt' in list_file:
    list_file.remove('0.null.txt')
    
list_file.sort()
    
len(list_file)

5

In [10]:
from tqdm import tqdm
from bs4 import BeautifulSoup
from lxml import etree
import re

In [11]:
for a in tqdm(range(len(list_file)), desc='Progress', ncols=77):
    with open('./part/' + list_file[a], 'r', encoding='UTF-8') as file:
        resp = file.read()
        
    # = = = = = = = = = = = = = = =
    
    soup = BeautifulSoup(resp, 'lxml')
    html = etree.HTML(str(soup))
    
    # = = = = = = = = = = = = = = =
    
    title = html.xpath('//h1[@class="product__title-link"]/text()')[0].strip()
    
    # = = = = = = = = = = = = = = =

    subtitle = html.xpath('//h1[@class="product__title-link"]/span[@class="product__subtitle"]/text()')
    if len(subtitle) == 0:
        subtitle = ''
    else:
        subtitle = subtitle[0].strip()
    
    # = = = = = = = = = = = = = = =
    
    artkl = re.sub('\n +', '\n', html.xpath('//div[@class="product__artkl"]/text()')[0].strip())
    
    # = = = = = = = = = = = = = = =
    
    product_id = html.xpath('//div[@class="product"]/@data-article-id')[0].strip()
    
    # = = = = = = = = = = = = = = =
    
    price = html.xpath('//div[@class="product__new-price"]/text()')[0].strip()
    
    # = = = = = = = = = = = = = = =
    
    oe = ';'.join([re.sub('^.*?OE-', '', oe.strip()).strip() for oe in html.xpath('//a[@class="product-oem__link"]/text()')])
    
    # = = = = = = = = = = = = = = =
    
    kit = '\n'.join([kit.strip() for kit in html.xpath('//div[@class="product__kit"]/ul/li/text()')])

    # = = = = = = = = = = = = = = =
    
    src = html.xpath('//div[contains(@class, "main-image")]/img/@src')[0].strip()
    
    # = = = = = = = = = = = = = = =
    
    maker_id = ';'.join([make.strip() for make in html.xpath('//div[@class="compatibility__maker-title"]/@data-maker-id')])

    # = = = = = = = = = = = = = = =
    
    list_tr = html.xpath('//table[@class="product__table"]/tr')
    
    # = = = = = = = = = = = = = = =
    
    list_li = html.xpath('//div[@class="product-analogs__wrapper"]/ul/li')

    # = = = = = = = = = = = = = = =
    
    df_temp = pd.DataFrame([{'No.': int(list_file[a].replace('.txt', '')),
                             'Title': title,
                             'Subtitle': subtitle,
                             'Artkl': artkl,
                             'Product_Id': product_id,
                             'Price': price,
                             'Vehicle_1': '',
                             'Vehicle_2': '',
                             'OE': oe,
                             'Kit': kit,
                             'Pic': '',
                             'Url': '',
                             'Src': src,
                             'Maker_Id': maker_id}])
    
    for tr in list_tr:
        name = tr.xpath('./td[1]/text()')[0].strip()
        if name[-1] == ':':
            df_temp.loc[0, name.upper()[:-1]] = tr.xpath('./td[2]/text()')[0].strip()
        else:
            df_temp.loc[0, name.upper()] = tr.xpath('./td[2]/text()')[0].strip()
    
    for li in list_li:
        name = li.xpath('./span/text()')[0].strip()
        if name[-1] == ':':
            df_temp.loc[0, name.lower()[:-1]] = ''.join(li.xpath('./text()')).strip()
        else:
            df_temp.loc[0, name.lower()] = ''.join(li.xpath('./text()')).strip()

    output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

    # = = = = = = = = = = = = = = =
    
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['No.'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./part.xlsx', index=False)

output_correct

Progress: 100%|████████████████████████████████| 5/5 [00:00<00:00, 29.80it/s]


,No.,Title,Subtitle,Artkl,Product_Id,Price,Vehicle_1,Vehicle_2,OE,Kit,...,blue print,profipower,PRODUKTIONSNUMMER,febi bilstein,lucas,metzger,pierburg,sidat,siemens,vemo
0,1,MASTER-SPORT 0951283-PCS-MS Leerlaufregler,elektrisch,Art. Nr: 0951283-PCS-MS\nHersteller: MASTER-SPORT,8722705,"7,72 €",,,9945635,,...,,,,,,,,,,
1,2,MASTER-SPORT 0971101-PCS-MS Leerlaufregler,elektrisch,Art. Nr: 0971101-PCS-MS\nHersteller: MASTER-SPORT,8725935,"9,24 €",,,1920 9V,,...,,,,,,,,,,
2,3,MASTER-SPORT 0951034-PCS-MS Leerlaufregler,elektrisch,Art. Nr: 0951034-PCS-MS\nHersteller: MASTER-SPORT,8722686,"9,96 €",,,77 02 217 296,,...,,,,,,,,,,
3,4,JPN 75E0032-JPN Leerlaufregler,elektrisch,Art. Nr: 75E0032-JPN\nHersteller: JPN,17799893,"9,96 €",,,92061898,,...,ADG074209,6922,,,,,,,,
4,5,KRAFT 9901532 Leerlaufregler,,Art. Nr: 9901532\nHersteller: KRAFT,19199505,"10,48 €",,,08 17 254;17 112 027;8 17 254,,...,,,K9901532,21160,FDB 954,0908002,7.05432.01.0,87.015,C2771,V40-77-0001
